In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("../dataset/240128_TripAdvisorHotelReviews/dataset_1K_removed_stopwords_ver.csv")

In [5]:
df['tokens'][0]

"['everyone', 'alby', 'made', 'reservation', 'deenny', 'greeted', 'every', 'morning', 'hailed', 'cabs', 'jennifer', 'front', 'desk', 'made', 'sure', 'stay', 'wonderful', 'already', 'rebooked', 'extended', 'stay', 'next', 'month', 'work', 'takes', 'back', 'thanks']"

In [16]:
##!! 저장을 해도 어차피 다시 list로 변환을 해주어야 한다. 

import pandas as pd
import ast

# 문자열을 파싱하여 리스트로 변환하는 함수
def parse_string_to_list(tokens_str):
    return ast.literal_eval(tokens_str)

# 모든 행에 대해 함수 적용
df['tokens'] = df['tokens'].apply(parse_string_to_list)



In [17]:
type(df['tokens'][0])

list

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 예시 데이터 (refined_df['tokens']에 토큰이 이미 들어가 있다고 가정)
# 토큰화된 데이터가 저장된 컬럼 이름이 'tokens'일 경우:
tokenized_text = df['tokens']

# 토크나이저 객체 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)

# 토큰들을 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(tokenized_text)

# 시퀀스 패딩
max_sequence_length = 200  # 시퀀스의 최대 길이 (임의 설정)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

2024-02-16 16:50:06.388507: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
len(padded_sequences[0])

200

In [19]:
sentences = df['tokens']

In [20]:
sentences[0]

['everyone',
 'alby',
 'made',
 'reservation',
 'deenny',
 'greeted',
 'every',
 'morning',
 'hailed',
 'cabs',
 'jennifer',
 'front',
 'desk',
 'made',
 'sure',
 'stay',
 'wonderful',
 'already',
 'rebooked',
 'extended',
 'stay',
 'next',
 'month',
 'work',
 'takes',
 'back',
 'thanks']

In [21]:
from gensim.models import Word2Vec

# 토큰화된 문장으로 Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)


In [22]:
# 단어 임베딩
embedding_dim = 300
embedded_sequences = [[model.wv[word] for word in sequence] for sequence in sentences]

In [34]:
print(type(embedded_sequences[0][0]))
print(embedded_sequences[0][0])


<class 'numpy.ndarray'>
[-2.27506249e-03 -1.43191323e-01  4.71233130e-01  3.45108062e-01
  2.04571024e-01 -1.63900316e-01 -9.73402709e-02  7.67093956e-01
  5.66698432e-01  6.94274232e-02 -7.17939734e-02 -1.43689126e-01
 -3.36179525e-01  6.65461481e-01 -3.28025967e-01 -4.64671493e-01
  2.42835134e-01 -2.75199831e-01  9.51434001e-02 -1.73462525e-01
 -1.82299227e-01 -9.82957259e-02  4.15712833e-01 -2.18232602e-01
  7.18213320e-02  4.49402511e-01 -6.60594940e-01  4.56836849e-01
 -3.86141658e-01 -2.12032683e-02 -8.51802975e-02  6.75134838e-01
  6.68359995e-02  1.51967201e-02 -6.79864168e-01 -8.58689845e-02
 -1.94605693e-01 -2.04200059e-01  3.47510964e-01 -6.41800687e-02
 -2.36679480e-04 -1.78667784e-01  1.01720020e-01 -5.44405937e-01
 -2.73129910e-01 -1.38759896e-01  2.42875010e-01 -1.56210944e-01
  2.91986810e-03  4.04177189e-01  6.79537132e-02  1.40970368e-02
  2.34983694e-02 -2.71483213e-01  3.88604581e-01 -5.67896813e-02
 -1.87130615e-01  2.36883879e-01  3.28505844e-01 -2.37209320e-01
 

In [35]:
# 패딩 처리
padded_sequences = pad_sequences(embedded_sequences, padding='post', maxlen=200)

In [40]:
len(padded_sequences[0][0])

#총 10000개의 데이터가 있고
#각 데이터는 200을 max length로 해서 맞춰 놓음. 
#그리고 각 단어별로는 300을 embedding 차원으로 정함. 

300

In [41]:
labels = df['ratings']
labels[0]

'high'

In [42]:
import numpy as np

# labels 배열
labels = np.array(labels)

# 값을 숫자로 변환하는 코드
labels = np.array([1 if value == 'high' else 0 for value in labels])

print("Modified labels:", labels)


Modified labels: [1 1 1 ... 0 0 0]


In [47]:
import numpy as np

# 넘파이 배열로 변환
embedded_sequences = np.array(padded_sequences)
labels = np.array(labels)


In [48]:
from sklearn.model_selection import train_test_split

# PyTorch DataLoader로 사용할 데이터셋 클래스를 정의하고 데이터를 생성하는 작업이 필요

# 예시: X는 embedded_sequence, y는 refined_df['ratings']
X_train, X_temp, y_train, y_temp = train_test_split(embedded_sequences, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [49]:
type(X_train)

numpy.ndarray

In [50]:
# 결과 확인
print("Train data shapes:", X_train.shape, y_train.shape)
print("Validation data shapes:", X_val.shape, y_val.shape)
print("Test data shapes:", X_test.shape, y_test.shape)

Train data shapes: (7000, 200, 300) (7000,)
Validation data shapes: (1500, 200, 300) (1500,)
Test data shapes: (1500, 200, 300) (1500,)


In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 데이터셋을 플랫하게 펼침
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_val.reshape(X_val.shape[0], -1)


In [53]:
X_train_flat.shape

(7000, 60000)

In [54]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# 모델 초기화
model = LogisticRegression()

# 모델 학습
model.fit(X_train_flat, y_train)

/Users/songye/opt/anaconda3/envs/aiffel2/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [55]:
# 예측
y_pred = model.predict(X_val_flat)

# 정확도 평가
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.7013333333333334
